In [4]:
import pandas as pd
import numpy as np
import tqdm
import re
import json

In [6]:
# KR-WordRank 설치
!pip install krwordrank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [7]:
from krwordrank.word import KRWordRank
from krwordrank.hangle import normalize
from krwordrank.word import KRWordRank

In [9]:
from google.colab import drive
drive.mount('/content/drive')
DATA_PATH = f'/content/drive/MyDrive/data/FPD'

Mounted at /content/drive


# RDASS 점수가 낮은 100개 데이터에 WORDRANK 추출 방법 적용해보기

In [19]:
df = pd.read_csv(f"{DATA_PATH}/mean_low_score.csv")

In [20]:
df.head(2)

,Unnamed: 0,title,blog_content,answer_tags,new_tags,RDASS_mean,content_len
0,774,[블책]전업주부 부수입 만들기 시작.,저는 그동안 좋아하는 일들만 했다면2023년은 잘 하는 일을 하면서 수익을 창출 해...,#블책 #부업하기 #부수입창출 #전업주부부업 #sns마케팅,#전업주부 #부수입 #SNS #육아 #틱톡,0.181967,1092
1,571,[밴쿠버 Vancouver 일상] 회사 송년식사 Wedgewood Hotel,몇일전부터 기대했던 회사 송년식사올해는 장소가#Wedgewood Hotel <end...,#밴쿠버맛집 #맛집 #캐나다 #밴쿠버 #크리스마스,#밴쿠버 #호텔 #코스요리 #디저트 #하얀눈,0.189568,911


In [23]:
# 텍스트 데이터 전처리 함수

def preprocessing(text):
    # 한글, 영문, 숫자만 남기고 모두 제거합니다.
    text = re.sub('[^가-힣ㄱ-ㅎㅏ-ㅣa-z ]', '', text)
    # 중복으로 생성된 공백값을 제거합니다.
    text = re.sub('[\s]+', ' ', text)
    # 영문자를 소문자로 만듭니다.
    text = text.lower()
    return text

In [24]:
df['blog_content'] = df['blog_content'].map(preprocessing)

In [25]:
df.head()

,Unnamed: 0,title,blog_content,answer_tags,new_tags,RDASS_mean,content_len
0,774,[블책]전업주부 부수입 만들기 시작.,저는 그동안 좋아하는 일들만 했다면년은 잘 하는 일을 하면서 수익을 창출 해야겠다는...,#블책 #부업하기 #부수입창출 #전업주부부업 #sns마케팅,#전업주부 #부수입 #SNS #육아 #틱톡,0.181967,1092
1,571,[밴쿠버 Vancouver 일상] 회사 송년식사 Wedgewood Hotel,몇일전부터 기대했던 회사 송년식사올해는 장소가edgewood otel end성급 호...,#밴쿠버맛집 #맛집 #캐나다 #밴쿠버 #크리스마스,#밴쿠버 #호텔 #코스요리 #디저트 #하얀눈,0.189568,911
2,843,"오늘의 루틴, 12월 22일",오늘의 핫 이슈endendend취득세 중과 완화주택 주택 법인 양도세 중과 배제 연...,#신문읽기 #공부하는엄마 #육아맘 #성공습관 #한국경제,#취득세 #양도세 #단기양도세율 #코로나 #감염증상,0.219899,857
3,679,종강 직후의 나날들,일기 일상 대학생 왠지 블로그 글을 오랜만에 적는 기분이다 사실 오랜만 맞음종강은 ...,#가타부타 #참치포케 #우니 #삼각지 #카페,#인턴 #삼각지 #영화관 #피그위스키센트밤 #종강,0.225414,1617
4,887,Dec: 즐거운 연말 시작 -♥,짠 end 유부초밥이 너무 땡겨서 도시락 싸가기 성공 end근데 학원 너무 추워서 ...,#빌라인아파트 #퀙스 #quew #구월동카페 #카페,#연말 #왕트리 #짝꿍집 #눈사람 #카페,0.231139,1275


In [27]:
# 문장 단위로 분리
df['sentence_lst'] = df['blog_content'].str.split('end')

In [28]:
# 나눠질때 na값으로 나눠진 데이터 채우기
df = df.fillna('')

In [30]:
# 문장 분리할때 공백이 발생하여 제거하는 함수 생성
def remove_blank(lst):
    return [v for v in lst if v]

In [29]:
# 공백 제거
df['sentence_lst'] = df['sentence_lst'].map(remove_blank)

# KR-WordRank를 사용한 키워드 추출<br>
출처_https://github.com/lovit/KR-WordRank

In [31]:
# 빈 리스트 생성
wr_list = []

In [34]:
# 1개의 게시물 먼저 확인해보기
min_count = 3   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
verbose = True
wordrank_extractor = KRWordRank(min_count, max_length , verbose)

beta = 0.85    # PageRank의 decaying factor beta
max_iter = 10

texts = df['sentence_lst'][0]
 
texts = [normalize(text,english=False , number=True) for text in texts ]
keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:15]:
        print('%8s:\t%.4f' % (word, r))

# remove stopwords

stopwords = {'그리고', '있었', '수지', '있어', '정도','제가','것을','것들','하는','인조를','이날은','빨리','나가','길에','남았던','타고','이런','갑자기','함께',"보냈어요","했다",'저희','월일','이날','보니'
            , '그리','맛있','정말','아주','그렇게','그래서','속에','님의','많은','있습니','좋은','이번에','벌써','많이','살면','이제','했는데','다시','같이','그렇지만','계속땡기는','사실','그래','있고'
            ,'하고','관한','위에','있는','또는','너무','오늘의','내가','이렇게','같다','마냥','그런','겪는','만든','저를','해서','저애','아니','다녀왔어요','바로','계속','다행히','여기','오늘','같은'
             ,'좋아','근데','왔던','먹는','끊어진','보는','저는','먹었','어제','이거','찍어','거의','이때','우리','없는','어느정도','그래도','한다','갔는','가서','나도','나는','그렇','위해','때가',
        '진짜','다른','아닌','역시','써야겠다는','있어서','았었','먹고','합니다','재밌','넘나','그냥','했더니','물어서','해야지','있다','하구','오랜만에','시킨','놓은','간신히','돌아오는길','만들어','지난주에'}
passwords = {word:score for word, score in sorted(
    keywords.items(), key=lambda x:-x[1])[:300] if not (word in stopwords)}

print('num passwords = {}'.format(len(passwords)))
print(passwords)
print(list(passwords.keys())[:5])


# wr_list.append(list(passwords.keys())[:5])
# wr_list

scan vocabs ... 
num vocabs = 90
done
      제가:	1.9822
      다른:	1.5681
      하는:	1.5476
      것들:	1.3857
      하고:	1.3658
      것을:	1.3273
      위에:	1.2596
      있는:	1.2292
     아이들:	1.1357
      교육:	1.1274
     조회수:	1.0606
     이야기:	1.0352
      좋아:	0.9777
      영상:	0.9758
      관한:	0.8915
num passwords = 11
{'아이들': 1.1357004480890254, '교육': 1.1273632453713391, '조회수': 1.0605867508370053, '이야기': 1.035154689691718, '영상': 0.9757642238591199, '일을': 0.8188513756541816, '리뷰': 0.8043849367809524, '게시': 0.7019294757858626, '사람들': 0.670910097326634, '육아': 0.5998405128083298, '생각': 0.5997743691761104}
['아이들', '교육', '조회수', '이야기', '영상']


In [35]:
# 파라미터 설정
min_count = 5   # 단어의 최소 출현 빈도수 (그래프 생성 시)
max_length = 10 # 단어의 최대 길이
verbose = False #True
wordrank_extractor = KRWordRank(min_count, max_length , verbose)

In [37]:
# for문으로 반복해서 전체 키워드 구하는 함수 만들기

def make_wr(x):
    beta = 0.85    # PageRank의 decaying factor beta
    max_iter = 10
    texts = x
    texts = [normalize(text,english=False , number=True) for text in texts ]
    keywords, rank, graph = wordrank_extractor.extract(texts, beta, max_iter)

    for word, r in sorted(keywords.items(), key=lambda x:x[1], reverse=True)[:10]:
            pass
            # print('%8s:\t%.4f' % (word, r))

    # remove stopwords

    stopwords = {'그리고', '있었', '수지', '있어', '정도'}
    passwords = {word:score for word, score in sorted(
            keywords.items(), key=lambda x:-x[1])[:300] if not (word in stopwords)}

    # print('num passwords = {}'.format(len(passwords)))
    # print(passwords)

    return list(passwords.keys())


In [38]:
make_wr(df['sentence_lst'][1])

[]

In [39]:
df['sentence_lst']

0     [저는 그동안 좋아하는 일들만 했다면년은 잘 하는 일을 하면서 수익을 창출 해야겠다...
1     [몇일전부터 기대했던 회사 송년식사올해는 장소가edgewood otel , 성급 호...
2     [오늘의 핫 이슈, 취득세 중과 완화주택 주택 법인 양도세 중과 배제 연장 단기양도...
3     [일기 일상 대학생 왠지 블로그 글을 오랜만에 적는 기분이다 사실 오랜만 맞음종강은...
4     [짠 ,  유부초밥이 너무 땡겨서 도시락 싸가기 성공 , 근데 학원 너무 추워서 밥...
                            ...                        
95    [크리스마스 이런 거 있으면짝꿍이 꼭 해보라고 알려주는데결과를 보면 이게 맞는 건가...
96    [월엔 항상 하고 싶은게 넘치지만 또 해야될 것도 넘쳐나아직 크리스마스 안 지났으니...
97    [지난주 금토 친구들과 함께 다녀온 워커힐호캉스 역시, 성은 좋구나 돈 많이 벌어야...
98    [귀염뽀쟉 예리미소희 보러 서울 간다tx 진짜 오랜만대학갈려고 처음 탔던거 기억나네...
99    [진정으로 권위가 있는 사람은 실수를 인정하는 것을 두려워하지 않는다버트런드 러셀권...
Name: sentence_lst, Length: 100, dtype: object

In [ ]:
# 적용하기

tqdm.pandas()
# df['wordRank'] = df['sentence_lst'].progress_map(make_wr)

  0%|          | 1/5068 [00:00<00:06, 812.53it/s]


AttributeError: ignored

In [ ]:
# 100개가 모두 완료되었는지 확인하기
len(wr_list)

100

In [ ]:
# wordrank 컬럼에 값 넣기
df['wordrank'] = wr_list

In [ ]:
# 결과 확인하기
df.head()

,Unnamed: 0,title,blog_content,answer_tags,new_tags,RDASS_mean,content_len,sentence_lst,wordrank
0,774,[블책]전업주부 부수입 만들기 시작.,저는 그동안 좋아하는 일들만 했다면년은 잘 하는 일을 하면서 수익을 창출 해야겠다는...,#블책 #부업하기 #부수입창출 #전업주부부업 #sns마케팅,#전업주부 #부수입 #SNS #육아 #틱톡,0.181967,1092,[저는 그동안 좋아하는 일들만 했다면년은 잘 하는 일을 하면서 수익을 창출 해야겠다...,"[많은, 보다, 조회수, 다른, 교육에]"
1,571,[밴쿠버 Vancouver 일상] 회사 송년식사 Wedgewood Hotel,몇일전부터 기대했던 회사 송년식사올해는 장소가edgewood otel end성급 호...,#밴쿠버맛집 #맛집 #캐나다 #밴쿠버 #크리스마스,#밴쿠버 #호텔 #코스요리 #디저트 #하얀눈,0.189568,911,"[몇일전부터 기대했던 회사 송년식사올해는 장소가edgewood otel , 성급 호...","[소고기, 스테이크, 캐나다, 호텔, 브로콜리]"
2,843,"오늘의 루틴, 12월 22일",오늘의 핫 이슈endendend취득세 중과 완화주택 주택 법인 양도세 중과 배제 연...,#신문읽기 #공부하는엄마 #육아맘 #성공습관 #한국경제,#취득세 #양도세 #단기양도세율 #코로나 #감염증상,0.219899,857,"[오늘의 핫 이슈, 취득세 중과 완화주택 주택 법인 양도세 중과 배제 연장 단기양도...","[재산세, 아이, 매입임대, 주택, 장기]"
3,679,종강 직후의 나날들,일기 일상 대학생 왠지 블로그 글을 오랜만에 적는 기분이다 사실 오랜만 맞음종강은 ...,#가타부타 #참치포케 #우니 #삼각지 #카페,#인턴 #삼각지 #영화관 #피그위스키센트밤 #종강,0.225414,1617,[일기 일상 대학생 왠지 블로그 글을 오랜만에 적는 기분이다 사실 오랜만 맞음종강은...,"[용산구, 종강, 시작, 인턴, 영화]"
4,887,Dec: 즐거운 연말 시작 -♥,짠 end 유부초밥이 너무 땡겨서 도시락 싸가기 성공 end근데 학원 너무 추워서 ...,#빌라인아파트 #퀙스 #quew #구월동카페 #카페,#연말 #왕트리 #짝꿍집 #눈사람 #카페,0.231139,1275,"[짠 , 유부초밥이 너무 땡겨서 도시락 싸가기 성공 , 근데 학원 너무 추워서 밥...","[일상, 짝꿍, 아주, 맛있, 진짜]"


In [ ]:
# 파일 저장하기
df.to_csv(f'{DATA_PATH}/word_rank.csv', index=False)